---
# Crash Course Python for Data Science - Intro to Pandas 
---
# 03 - Cleaning Data 
---



### Cleaning data is an *integral part of being a data scientist*  
In most contexts, the data you'll be using to generate insights will not be perfect. Far from it. There will be missing values. There will be incorrect values. There could be duplicate values. Columns could have data types that break your code. There could be outliers that mess things up. You may even need to merge data from different sources to do a thorough analysis. Fortunately, pandas has some really powerful methods for cleaning your data. Which is good news, because most data scientists tend to spend the bulk of their time cleaning data before they can analyze it.  

*In other words, you clean your data in order to prepare it for analysis and model-building.*

### Begin by importing your tools

In [94]:
# Let's begin by importing pandas
import pandas as pd
print('pandas imported!')

pandas imported!


## Reading in your data

In [95]:
# We use the pandas method pandas.read_csv("filepath") to create a DataFrame 
# and assign it to a variable:

df = pd.read_csv("https://raw.githubusercontent.com/axrd/datasets/master/gdpmessy.csv", index_col=0)

## Quick inspection of the data

In [96]:
# We'll use the head() method, which by default prints out the first 5 rows. I 
# want to look at the first 10 rows:
df.head(10)

,COUNRY,gdp,CODE
0,Afghanistan,21.71,AFG
1,Albania,13.40,ALB
2,Algeria,227.80,NaN
3,American Samoa,0.75,ASM
4,Andorra,NaN,AND
5,NaN,131.40,AGO
6,Anguilla,0.18,AIA
7,Antigua and Barbuda,1.24,ATG
8,Argentina,536.20,ARG
9,Armenia,10.88,ARM


In [97]:
# Then the tail method. Also for 10 values.
df.tail(10)

,COUNRY,gdp,CODE
212,Uruguay,55.60,URY
213,Uzbekistan,63.08,UZB
214,Vanuatu,8200.00,VUT
215,Venezuela,209.20,VEN
216,Vietnam,187.80,VNM
217,Virgin Islands,5.08,VGB
218,West Bank,6.64,WBG
219,Yemen,45.45,YEM
220,Zambia,25.61,ZMB
221,Zimbabwe,13.74,ZWE


In [98]:
# We're supposed to have 222 observations (rows) and 3 columns. Let's check:

df.shape

(222, 3)

In [99]:
# A great way to get an overview of our data:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 222 entries, 0 to 221
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   COUNRY  221 non-null    object 
 1   gdp     219 non-null    float64
 2   CODE    220 non-null    object 
dtypes: float64(1), object(2)
memory usage: 6.9+ KB


**We've got some missing values! We'll address that in a minute. First, let's fix our duplicate index and rename our columns something useful and consistent.**

## Duplicate index
**Remember how we dealt with this a few exercises ago, as we were learning about loading data?**



In [100]:
df.columns

Index(['COUNRY', 'gdp', 'CODE '], dtype='object')

In [101]:
# We need to drop the first column (axis = 1) and do it in place:
df.drop(index = 0, axis=1, inplace=True)

In [102]:
df.head()

,COUNRY,gdp,CODE
1,Albania,13.40,ALB
2,Algeria,227.80,NaN
3,American Samoa,0.75,ASM
4,Andorra,NaN,AND
5,NaN,131.40,AGO


## Column names

**We also covered how to rename columns. But we only showed you how to rename one at a time:**  


*   `COUNRY` should be spelled correctly
*   `gdp` is lowercase and should also communicate scale
*   `CODE` has an trailing space in it  


**Here's how we'd do it one by one:**

`df.rename(columns={'COUNRY':'COUNTRY'}, inplace=True)`  
`df.rename(columns={'gdp':'GDP (BILLIONS)'}, inplace=True)`  
`df.rename(columns={'CODE ':'CODE'}, inplace=True)`

**But that's not Pythonic. Let's try and change all of these column names at once.**


In [103]:
# Pandas has a way to select columns because it treats them as an index object:
df.columns

# Notice that "CODE " has an empty space after the last character. This happens!

Index(['COUNRY', 'gdp', 'CODE '], dtype='object')

In [104]:
df['CODE ']

1      ALB
2      NaN
3      ASM
4      AND
5      AGO
      ... 
217    VGB
218    WBG
219    YEM
220    ZMB
221    ZWE
Name: CODE , Length: 221, dtype: object

In [105]:
df.columns = ["COUNTRY", "GDP_BILLIONS", "CODE"]

In [106]:
df.columns

# Done.

Index(['COUNTRY', 'GDP_BILLIONS', 'CODE'], dtype='object')

In [107]:
list(df.columns)

['COUNTRY', 'GDP_BILLIONS', 'CODE']

In [108]:
df.head()

,COUNTRY,GDP_BILLIONS,CODE
1,Albania,13.40,ALB
2,Algeria,227.80,NaN
3,American Samoa,0.75,ASM
4,Andorra,NaN,AND
5,NaN,131.40,AGO


In [109]:
df['GDP_BILLIONS']

1       13.40
2      227.80
3        0.75
4         NaN
5      131.40
        ...  
217      5.08
218      6.64
219     45.45
220     25.61
221     13.74
Name: GDP_BILLIONS, Length: 221, dtype: float64

## Missing values?
We noticed immediately that we had some missing values to deal with. Every feature has at least one missing value (nulls).

In [110]:
# Pandas has a nifty tool to check for null values. For now, you can think of 
# nulls as missing values. 

df.isnull()

,COUNTRY,GDP_BILLIONS,CODE
1,False,False,False
2,False,False,True
3,False,False,False
4,False,True,False
5,True,False,False
...,...,...,...
217,False,False,False
218,False,False,False
219,False,False,False
220,False,False,False


In [111]:
# Ok, but scrolling throuh 222 rows looking for "True" sounds awful. There has 
# to be a better way, right? There is. Remember the sum function?

df.isnull().sum()

# Six targets. Let's take them one at a time. Starting with "Country".

COUNTRY         1
GDP_BILLIONS    3
CODE            2
dtype: int64

In [112]:
# We'll subset our data by choosing ALL the rows that have missing values.

df[df.isnull().any(axis=1)]

,COUNTRY,GDP_BILLIONS,CODE
2,Algeria,227.80,NaN
4,Andorra,NaN,AND
5,NaN,131.40,AGO
10,Aruba,NaN,ABW
13,Azerbaijan,77.91,NaN
20,Belize,NaN,BLZ


**Let's say we were able to identify the actual values that belong in the DataFrame as follows:**



*   Algeria has an Alpha-3 code of DZA
*   Andorra has a GDP of 4.80 billion
*   Azerbaijan has an Alpha-3 code of AZE
*   Aruba has a GDP of 2.52 billion
*   Belize has a GDP of 1.67 billion
*   Angola is the missing 'COUNTRY' value  


We'll use the pandas [at() method to handle this.](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.at.html)







In [113]:
# We specify the row index, the column name, and the value we'd like to assign to it:
df.at[2, "CODE"] = "DZA"
df.at[4, "GDP_BILLIONS"] = 4.80
df.at[5, "COUNTRY"] = "Angola"
df.at[10, "GDP_BILLIONS"] = 2.52
df.at[13, "CODE"] = "AZE"
df.at[20, "GDP_BILLIONS"] = 1.67

In [114]:
df.isnull().sum()

COUNTRY         0
GDP_BILLIONS    0
CODE            0
dtype: int64

In [115]:
df.head(10)

,COUNTRY,GDP_BILLIONS,CODE
1,Albania,13.40,ALB
2,Algeria,227.80,DZA
3,American Samoa,0.75,ASM
4,Andorra,4.80,AND
5,Angola,131.40,AGO
6,Anguilla,0.18,AIA
7,Antigua and Barbuda,1.24,ATG
8,Argentina,536.20,ARG
9,Armenia,10.88,ARM
10,Aruba,2.52,ABW


## Less obvious things:
We checked for missing values, but now we need to worry about duplicate or incorrect values. Let's look for duplicates.  

We'll use the `value_counts()` method, sorted to see where any values appear more than once. The `head()` at the end of the chain limits our output to five observations to keep things manageable. If any of them have multiple values beyond the first five observations, we'll expand the head until we account for all duplicates. 

In [116]:
df["COUNTRY"].value_counts(sort=True).head()

COUNTRY
Albania          1
Palau            1
Netherlands      1
New Caledonia    1
New Zealand      1
Name: count, dtype: int64

That one's good to go. No duplicates (we wouldn't expect any here).

In [117]:
df["GDP_BILLIONS"].value_counts(sort=True).head(5)

GDP_BILLIONS
0.18     3
0.75     2
0.16     2
13.40    1
44.69    1
Name: count, dtype: int64

Ok. Several duplicates. But we'd expect some in GDP since we're using billions and only two decimal places. In terms of preparing our data, next steps here would be to double-check those values with official records. Alternatively, we could add decimal places for greater precision. 

In [118]:
df["CODE"].value_counts(sort=True).head()

CODE
VGB    2
HKG    2
MOZ    1
NPL    1
NLD    1
Name: count, dtype: int64

This one's dangerous. Like `COUNTRY` we wouldn't expect any duplicate values here, but we've got two. Let's see what's going on:

In [119]:
df[df['CODE'] == "VGB"]

,COUNTRY,GDP_BILLIONS,CODE
28,British Virgin Islands,1.10,VGB
217,Virgin Islands,5.08,VGB


So there's the British Virgin Islands and the American Virgin Islands. The latter has an A-3 code of VIR, not VGB. Let's fix it:

In [120]:
df.at[217, "CODE"] = "VIR"

In [121]:
df[df['CODE'] == "HKG"]

,COUNTRY,GDP_BILLIONS,CODE
89,Hong Kong,292.7,HKG
90,Hungary,129.7,HKG


Hungary is supposed to be HUN. Let's change it: 

In [122]:
df.at[90, "CODE"] = "HUN"

To be thorough, let's make sure our changes didn't create any new duplicates in the `CODE` column:

In [123]:
df["CODE"].value_counts(sort=True).head()

CODE
ALB    1
PLW    1
NLD    1
NCL    1
NZL    1
Name: count, dtype: int64

## Even less obvious things

In [124]:
df.sort_values(by='GDP_BILLIONS', ascending=False).head(10)

,COUNTRY,GDP_BILLIONS,CODE
211,United States,17420.0,USA
42,China,10360.0,CHN
214,Vanuatu,8200.0,VUT
101,Japan,4770.0,JPN
75,Germany,3820.0,DEU
70,France,2902.0,FRA
210,United Kingdom,2848.0,GBR
27,Brazil,2244.0,BRA
99,Italy,2129.0,ITA
164,Russia,2057.0,RUS


Clearly, Vanuatu isn't the 3rd largest economy in the world. Its GDP is actually supposed to be 0.82 billion.

In [125]:
df.at[214, "GDP_BILLIONS"] = 0.82

In [126]:
df[df['CODE'] == 'BGD']

,COUNTRY,GDP_BILLIONS,CODE
16,Bangladesh,186.6,BGD




---


## That was a lot of new stuff. Remember to practice with the exercise!